In [1]:
# TODO: Create PriorBox
import pickle
import numpy as np

In [2]:
def create_prior_boxes(box_configs, img_size, variances):
    img_width, img_height = img_size
    boxes_datas = []
    for layer_config in box_configs:
        boxes_data = []
        layer_width = layer_config['layer_width']
        layer_height = layer_config['layer_height']

        num_priors = layer_config['num_prior']
        aspect_ratios = layer_config['aspect_ratios']

        min_size = layer_config['min_size']
        max_size = layer_config['max_size']

        step_x = float(img_width) / float(layer_width)
        step_y = float(img_height) / float(layer_height)
        # series center of boxes
        linx = np.linspace(0.5 * step_x, img_width - 0.5 * step_x, layer_width)
        liny = np.linspace(0.5 * step_y, img_height - 0.5 * step_y, layer_height)

        centers_x, centers_y = np.meshgrid(linx, liny)
        centers_x = centers_x.reshape(-1, 1)
        centers_y = centers_y.reshape(-1, 1)
        # each element is (x,y) location with shape of (n,1)
        prior_boxes = np.concatenate((centers_x, centers_y), axis=1)
        # tile shape to (n, 2 * num_priors)
        prior_boxes = np.tile(prior_boxes, (1, 2 * num_priors))

        box_widths = []
        box_heights = []
        # traverse each of aspect ratios
        # then get corresponding width and height of each boxes
        for ar in aspect_ratios:
            if ar == 1 and len(box_widths) == 0:
                box_widths.append(min_size)
                box_heights.append(min_size)
            elif ar == 1 and len(box_widths) > 0:
                box_widths.append(np.sqrt(min_size * max_size))
                box_heights.append(np.sqrt(min_size * max_size))
            elif ar != 1:
                box_widths.append(min_size * np.sqrt(ar))
                box_heights.append(min_size / np.sqrt(ar))
        # because we need to get corner location by center
        # so we have to half of length
        box_widths = 0.5 * np.array(box_widths)
        box_heights = 0.5 * np.array(box_heights)

        # min_x
        prior_boxes[:, ::4] -= box_widths
        # min_y
        prior_boxes[:, 1::4] -= box_heights
        # max_x
        prior_boxes[:, 2::4] += box_widths
        # max_y
        prior_boxes[:, 3::4] += box_heights
        # normalize to [0-1]
        prior_boxes[:, ::2] /= img_width
        prior_boxes[:, 1::2] /= img_height

        prior_boxes = prior_boxes.reshape(-1, 4)
        # clip to 0-1
        prior_boxes = np.minimum(np.maximum(prior_boxes, 0.0), 1.0)
        prior_variances = np.tile(variances, (len(prior_boxes), 1))
        boxes_data = np.concatenate((prior_boxes, prior_variances), axis=1)
        boxes_datas.append(boxes_data)
        
    return np.concatenate(boxes_datas, axis=0)

In [3]:
def save_prior_boxes(box_configs, img_size, variances, path):
    boxes_priors = create_prior_boxes(box_configs, img_size, variances)
    pickle.dump(boxes_priors, open(path, 'wb'))

In [4]:
# test it
box_configs = [
    {'layer_width': 38, 'layer_height': 38, 'num_prior': 3, 'min_size':  30.0,
     'max_size': None, 'aspect_ratios': [1.0, 2.0, 1/2.0]},
    {'layer_width': 19, 'layer_height': 19, 'num_prior': 6, 'min_size':  60.0,
     'max_size': 114.0, 'aspect_ratios': [1.0, 1.0, 2.0, 1/2.0, 3.0, 1/3.0]},
    {'layer_width': 10, 'layer_height': 10, 'num_prior': 6, 'min_size': 114.0,
     'max_size': 168.0, 'aspect_ratios': [1.0, 1.0, 2.0, 1/2.0, 3.0, 1/3.0]},
    {'layer_width':  5, 'layer_height':  5, 'num_prior': 6, 'min_size': 168.0,
     'max_size': 222.0, 'aspect_ratios': [1.0, 1.0, 2.0, 1/2.0, 3.0, 1/3.0]},
    {'layer_width':  3, 'layer_height':  3, 'num_prior': 6, 'min_size': 222.0,
     'max_size': 276.0, 'aspect_ratios': [1.0, 1.0, 2.0, 1/2.0, 3.0, 1/3.0]},
    {'layer_width':  1, 'layer_height':  1, 'num_prior': 6, 'min_size': 276.0,
     'max_size': 330.0, 'aspect_ratios': [1.0, 1.0, 2.0, 1/2.0, 3.0, 1/3.0]},
]
img_size = (300, 300)
variances = [0.1, 0.1, 0.2, 0.2]
path = 'prior_boxes_own.pkl'
save_prior_boxes(box_configs, img_size, variances, path)